## Description:
Data wrangling & cleaning to compile a dataframe of all Pitchfork review/reviewer data and Spotify feature analysis to proceed with project's next steps (i.e. modeling reviewer bias, creating a recommendation engine based on these models).

The final output of this notebook is a dataframe combining Spotify analysis & Pitchfork album data (<b>'df_pitchfork_spotify'</b>) and a dataframe of reviewer index and the reviews they authored (<b>'authors_reviews'</b>).

<b>df_pitchfork_spotify</b>:

- <b>album</b> <i>(str)</i>: album name
- <b>artist</b> <i>(str)</i>: album artist
- <b>reviewid</b> <i>(int)</i>: review id - for album's Pitchfork review
- <b>spotify_album_id</b> <i>(str)</i>: album id - for obtaining its info from Spotify's API
- <b>album_tracks</b> <i>(list)</i>: album's track list
- <b>album_features</b> <i>(list of dictionaries)</i>: album's track's features; where dict key = track index, value = track's feature analysis scores
- <b>album_features_avg</b> <i>(dict)</i>: album's track's features, averaged; where dict key = feature, value = feature score

<b>df_authors_reviews</b>:
- <b>author_fullname</b> <i>(str)</i>: author's (reviewer's) full name
- <b>reviewid</b> <i>(list)</i>: list of id's of all reviews written by reviewer


## Contents:
#### 1. Wrangle Pitchfork data to index reviews by reviewer
- 1.1 Prepare data from Pitchfork SQL database for analysis
- 1.2 Clean reviewer list to avoid duplicating authors and misspellings
- 1.3 Generate dataframe, where reviews indexed by reviewers

#### 2. Pair Spotify API identifiers with review/reviewer data
- 2.1 Wrangle & clean Pitchfork data in preparation for appending Spotify data
- 2.2 Obtain Spotify API ID for each album
- 2.3 Extract Spotify album feature analysis


In [2]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

## 1. Wrangle Pitchfork data to index reviews by reviewer (output = 'authors_reviews' dataframe)

### 1.1. Prepare data from Pitchfork SQL database for analysis

In [8]:
engine = create_engine('sqlite:///database_pitchfork.sqlite') # Create database engine for Pitchfork SQL database
con = engine.connect() # Connect to engine

rs = con.execute('SELECT * FROM reviews') # Query the database (for reviews)

df_pitchfork = pd.DataFrame(rs.fetchall()) # Save query results to DataFrame
df_pitchfork.columns = rs.keys() # Name columns
con.close() # Close connection

df_pitchfork.head() # 'df_pitchfork': all data from Pitchfork SQL database, for general overview of data

,reviewid,title,artist,url,score,best_new_music,author,author_type,pub_date,pub_weekday,pub_day,pub_month,pub_year
0,22703,mezzanine,massive attack,http://pitchfork.com/reviews/albums/22703-mezz...,9.3,0,nate patrin,contributor,2017-01-08,6,8,1,2017
1,22721,prelapsarian,krallice,http://pitchfork.com/reviews/albums/22721-prel...,7.9,0,zoe camp,contributor,2017-01-07,5,7,1,2017
2,22659,all of them naturals,uranium club,http://pitchfork.com/reviews/albums/22659-all-...,7.3,0,david glickman,contributor,2017-01-07,5,7,1,2017
3,22661,first songs,"kleenex, liliput",http://pitchfork.com/reviews/albums/22661-firs...,9.0,1,jenn pelly,associate reviews editor,2017-01-06,4,6,1,2017
4,22725,new start,taso,http://pitchfork.com/reviews/albums/22725-new-...,8.1,0,kevin lozano,tracks coordinator,2017-01-06,4,6,1,2017


##### Column metadata:
<b>'reviewid'</b>: Review unique key<br>
<b>'title'</b>: Album title<br>
<b>'artist'</b>: Name of recording artist<br>
<b>'url'</b>: Review URL<br>
<b>'score'</b>: Score on 10-point scale<br>
<b>'best_new_music'</b>: Whether review categorized as a "Best New Music" review (1 for yes, 0 for no)<br>
<b>'author'</b>: Name of reviewer<br>
<b>'author_type'</b>: Reviewer type. Staff are given their specific titles while contributors are listed as "Contributor"<br>
<b>'pub_date'</b>: Review publication date<br>
<b>'pub_weekday'</b>: Review day of week<br>
<b>'pub_day'</b>: Review day of

### 1.2 Clean reviewer list for misspellings/duplicates and special characters

####  Create dictionary ('dict_dupes') for duplicate and misspelled names to replace w/ "correct" name

In [9]:
# key: misspelling/duplicate, value: replacement name (i.e. "correct" name)
dict_dupes = dict({'mike':'michael', 
                   'matt ':'matthew ', 
                   'chris ':'christopher ', 
                   'dr.':'', 
                   'fennessy':'fennessey', 
                   'andrew':'drew', 
                   'abby':'abigail', 
                   'linhardt':'lindhart', 
                   'alex ':'alexander ', 
                   'josh ':'joshua ', 
                   'bill':'william', 
                   'savy':'saby', 
                   'richard-san':'richardson', 
                   'ben ':'benjamin ', 
                   'nick':'nicholas ', 
                   'iii':'' })

#### Replace other special characters (ex. "mu") into form that Spotify API will recognize

In [10]:
dict_artist_correct = {'-ziq':'u-ziq'}

#### Data cleaning: 
- Remove special characters & punctuation
- Replace misspellings & nicknames w/ "correct" entries (from 'dict_dupes')
- Only include primary author

In [11]:
import re

authors = df_pitchfork['author']
artists = df_pitchfork['artist']

authors_new = []
artists_new = []

pattern_1 = r"(?<!\d)[.,;:](?!\d)" # remove all punctuation from string
pattern_2 = r'"' # remove double quotations

for index in range(len(df_pitchfork)):
    
    a = df_pitchfork.loc[index]['author']
    b = df_pitchfork.loc[index]['artist']
    
    artist_split = b.split(',')
    if len(artist_split) > 1:
        b = artist_split[0]
        
    for incorrect, correct in dict_dupes.items(): 
        if incorrect in a:
            a = re.sub(incorrect, correct, a)
    
    # remove punctuation        
    a = re.sub(pattern_1, "", a) 
    a = re.sub(pattern_2, "", a)
    
    # remove multiple authors to only include primary author
    a_split = a.split()
    if len(a_split) > 3: 
        a = a_split[0] + ' ' + a_split[1]
    
    # replace special characters in artists w/ something Spotify search can recognize
    for incorrect, correct in dict_artist_correct.items():
        if b == incorrect:
            b = re.sub(incorrect, correct, b)
    
    authors_new.append(a)
    artists_new.append(b)

df_pitchfork['artist'] = artists_new

#### Remove middle names/initials to avoid duplicating authors

In [12]:
first_name = [] 
last_name = [] 
full_name = []

for author in authors_new:
    
    author_split = author.split()
    
    if len(author_split) <= 3: # remove middle names & initials
        first = author_split[0]
        last = author_split[-1]
    else:
        firstlast = re.search('(.*) (.*)', author)
        first = firstlast.group(1)
        last = firstlast.group(2)
    
    first_name.append(first)
    last_name.append(last)
    
    full_name.append(first + " " + last)

##### (Optional:) Save as csv in case want to manually review  author list

In [131]:
from collections import OrderedDict

authors_ordered = pd.DataFrame(OrderedDict({'Last Name': last_name, 'First Name': first_name}))
authors_ordered.to_csv('authors_ordered2.csv')

#### "Pickle" cleaned Pitchfork database to use in other scripts

In [130]:
df_pitchfork.to_pickle('df_pitchfork.pkl')

### 1.2 Combine reviewers and reviews into dataframe ('authors_reviews'), where .index = reviewers, .values = reviews

In [71]:
df_authors_reviews = pd.DataFrame({'reviewid': df_pitchfork['reviewid'], 'author_fullname':full_name})
df_authors_reviews.groupby('author_fullname')
df_authors_reviews = pd.DataFrame(df_authors_reviews.groupby(['author_fullname'])['reviewid'].apply(list))
df_authors_reviews = pd.DataFrame({'reviews':df_authors_reviews['reviewid']}).reset_index()
df_authors_reviews.head()

,author_fullname,reviews
0,aaron leitko,"[21950, 21820, 21753, 21622, 21572, 21283, 213..."
1,abigail covington,"[22138, 21544]"
2,abigail garnett,"[21050, 20968, 20694, 20570, 20405, 20134, 199..."
3,adam dlugacz,[6794]
4,adam moerder,"[13207, 13007, 12956, 12939, 12861, 12673, 126..."


#### Save cleaned authors & reviews dataframe as csv

In [131]:
df_authors_reviews.to_pickle('df_authors_reviews.pkl')

## 2. Pair Spotify API identifiers with review/reviewer data

Spotify API access provided by Python library 'Spotipy' (https://github.com/plamere/spotipy). In order to access Spotify's API (with or without Spotipy), you must obtain a client ID & secret ID after registering an "app". Instructions here:  https://beta.developer.spotify.com/documentation/web-api/quick-start/

Before running, need to manually enter the following into the code below:
- Client ID
- Secret ID
- Redirect address (used to provide access token)
- Scope of query (https://beta.developer.spotify.com/documentation/general/guides/scopes/)

In [455]:
# Manually enter client id, secret id, and redirect address (used to provide access token)
myClientId = ''
mySecret = ''
myRedirect = '' # usually 'http://localhost:8888/callback'

# Scope of your query to Spotify API
scope = 'user-library-read' # ex: 'user-library-read'

### 2.1 Wrangle & clean Pitchfork data in preparation for appending Spotify data

In [140]:
# Generate dataframe of artist, album, and its review id
df1 = pd.DataFrame({'artist':df_pitchfork['artist'], 
                    'album':df_pitchfork['title'], 
                    'reviewid':df_pitchfork['reviewid'],
                    'score':df_pitchfork['score']})

#### Remove albums with artist = 'various artists'

In [141]:
df1 = df1[df1['artist']!='various artists'].reset_index(drop=True)

#### Remove 'EP' from EP album title (since it interferes w/ Spotify album search)

In [142]:
for index, row in df1.iterrows():
    album_split = row['album'].split()
    if len(album_split) == 0: # some Pitchfork database entries have empty title & artist, even though they have a reviewid
        continue
    if album_split[-1] == 'ep':
        album_new = " ".join(album_split[:-1])
        df1.at[index, 'album'] = album_new

### 2.2 Obtain Spotify API ID for each album

In [143]:
import sys
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials
from pandas.io.json import json_normalize

#### Set up access to Spotify API

In [144]:
if len(sys.argv) > 1:
    username = sys.argv[1]
else:
    print( 'Usage: %s username' % (sys.argv[0],))
    sys.exit()

# Generate access token
token = util.prompt_for_user_token(username, scope, myClientId, mySecret, myRedirect)

# Generate Spotify object w/ access token
client_credentials_manager = SpotifyClientCredentials(client_id = myClientId, client_secret = mySecret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

#sp = spotipy.Spotify(auth=token)
sp.trace = False

print(token)

#### Extract Spotify album ID's

In [145]:
import time

results_id = []
counter = 0

for index, row in df1.iterrows():
    
    # avoid hitting Spotify rate limit by pausing queries every 2000 queries
    counter = counter + 1
    if counter%2000 == 0: 
        time.sleep(120) # (sec)
    
    album = row['album']
    artist_multiple = row['artist'].split(',')
    artist = artist_multiple[0]

    # find album by name (documentation on queries here: https://beta.developer.spotify.com/documentation/web-api/reference/search/search/)
    results = sp.search(q = 'album:' + album + ' artist:' + artist, type = 'album', limit=20)
    
    # get results from album search
    if len(results['albums']['items']) == 0: # Spotify doesn't have the album
        narrow_result_id = np.nan
        results_id.append(narrow_result_id)    
        continue
    else:
        album_data = results['albums']['items']

    # narrow results to match artist, following album search
    df_results = json_normalize(album_data, 'artists')
    df_results['name'] = df_results['name'].str.lower() # all initial results
    df_narrow = df_results[df_results['name']==artist] # narrowed results by artist query
    
    # no results from search. so try second artist if >1 artist for album
    if len(df_narrow) == 0 & len(artist_multiple) > 1:
        df_results = json_normalize(album_data, 'artists')
        df_results['name'] = df_results['name'].str.lower() # all initial results
        df_narrow = df_results[df_results['name']==artist] # narrowed results by artist query
    
    # give up search: Spotify probably doesn't carry the album
    if len(df_narrow) == 0: 
        narrow_result_id = np.nan
        results_id.append(narrow_result_id)
        continue

    # extract necessary spotify album id from narrowed result
    if len(df_results.index) > len(album_data): # special case when narrowed result album has 2 authors; requires special parsing
        album_idx = 0
    else:
        album_idx = df_narrow.index[0]
    
    album_id = album_data[album_idx]['id']
    results_id.append(album_id)

print('done')

retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...2secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...2secs
retrying ...3secs
done


#### Append Pitchfork artist-album data with Spotify album identifiers

In [147]:
df1['spotify_album_id'] = results_id
df1.head()

,album,artist,reviewid,score,spotify_album_id
0,mezzanine,massive attack,22703,9.3,49MNmJhZQewjt06rpwp6QR
1,prelapsarian,krallice,22721,7.9,264k3RcZ8BGWTc1TlH8gvs
2,all of them naturals,uranium club,22659,7.3,5M3Cx38HT8sD8ulnrSZsXi
3,first songs,kleenex,22661,9.0,NaN
4,new start,taso,22725,8.1,7hhPEiSLGSFdHJEheB8IL3


### 2.3 Extract Spotify album feature analysis

Spotify feature analysis is conducted on a per-track basis (i.e. not album). Therefore, to obtain 'feature analysis' for an entire album, I average the album's tracks' features.

#### Drop albums w/ no Spotify data, since they contain no audio feature data for analysis.

There are a few albums in the Pitchfork database with reviewid's (and presumably reviews), but no album or artist information. Therefore, these are dropped as no Spotify data can be obtained.

In [37]:
df2 = df1
df2.dropna(subset=['spotify_album_id'], inplace=True)
df2 = df1.reset_index(drop=True)
df2.head()

,album,artist,reviewid,score,spotify_album_id
0,mezzanine,massive attack,22703,9.3,49MNmJhZQewjt06rpwp6QR
1,prelapsarian,krallice,22721,7.9,264k3RcZ8BGWTc1TlH8gvs
2,all of them naturals,uranium club,22659,7.3,5M3Cx38HT8sD8ulnrSZsXi
3,new start,taso,22725,8.1,7hhPEiSLGSFdHJEheB8IL3
4,stillness in wonderland,little simz,22704,7.1,4G50FUTTI4fCDyrxP1UEer


#### Get album tracks and feature analysis for each track

In [138]:
album_tracks = []
album_features = []
album_features_avg = []

album_date = []
album_trackcount = []
album_popularity = []
album_genre = []

for index, row in df2.iterrows(): # each row = album
    
    # avoid hitting Spotify rate limit by pausing queries every 2000 queries
    counter = counter + 1
    if counter%2000 == 0: 
        time.sleep(120) # (sec)  
        
    # 1. extract track list from album
    album_id = row['spotify_album_id']
    track_list = sp.album_tracks(album_id)['items']

    # 2. extract track name and Spotify feature analysis per track
    track_names = [] # list of names of all tracks in album
    track_features = [] # list of dictionaries, where each dictionary = feature scores for each track of album
    track_features_avg = [] # list of feature scores for album, averaged
    track_uris = []
    
    for track in track_list:
        track_names.append(track['name'])
        track_uris.append(track['uri'] )
    
    track_features = sp.audio_features(track_uris)
    
    if None in track_features:
        album_tracks.append(np.nan)
        album_features.append(np.nan)
        album_features_avg.append(np.nan)
        continue
    
    # 3. strip out info that is not an audio feature (ex. 'analysis_url', 'track_href', 'track uri')
    feature_list = []
    for key, value in track_features[0].items():
        if type(value) == float or type(value) == int:
            feature_list.append(key)
        else:
            continue
    
    # 4. average all tracks' features for the album
    track_features_avg = {}
    for feature in feature_list:
        to_average = []
        for feature_dict in track_features: # feature_dict = dictionary of features for the track 
            to_average.append(feature_dict[feature])
        average = sum(to_average) / len(to_average)
        track_features_avg[feature] = average # key: feature, value: averaged feature between all tracks of the album
    
    # 5. extract album info
    album_info = sp.album(album_id)
    
    # 6. compile album data
    
    album_tracks.append(track_names)
    album_features.append(track_features)
    album_features_avg.append(track_features_avg)
    
    album_date.append(album_info['release_date'])
    album_trackcount.append(len(track_list))
    album_popularity.append(album_info['popularity'])
    album_genres.append(album_info['genres'])

print('done')

done


#### Append newly extracted Spotify album feature analysis to final output ('df_pitchfork_spotify')

In [126]:
df2['album_tracks'] = album_tracks
df2['album_features'] = album_features
df2['album_features_avg'] = album_features_avg
df2['album_trackcount'] = album_trackcount
df2['album_popularity'] = album_popularity
df2['album_genres'] = album_genres
df2.head()

,album,artist,reviewid,score,spotify_album_id,album_tracks,album_features,album_features_avg,album_trackcount,album_popularity,album_genres
0,mezzanine,massive attack,22703,9.3,49MNmJhZQewjt06rpwp6QR,"[Angel, Risingson, Teardrop, Inertia Creeps, E...","[{'danceability': 0.714, 'energy': 0.309, 'key...","{'danceability': 0.6041818181818182, 'energy':...",11,64,[]
1,prelapsarian,krallice,22721,7.9,264k3RcZ8BGWTc1TlH8gvs,"[Transformation Chronicles, Hate Power, Confla...","[{'danceability': 0.108, 'energy': 0.806, 'key...","{'danceability': 0.131, 'energy': 0.867, 'key'...",4,10,[]
2,all of them naturals,uranium club,22659,7.3,5M3Cx38HT8sD8ulnrSZsXi,"[Introduction, That Clown's Got A Gun, The Lot...","[{'danceability': 0.613, 'energy': 0.85, 'key'...","{'danceability': 0.44499999999999995, 'energy'...",8,28,[]
3,new start,taso,22725,8.1,7hhPEiSLGSFdHJEheB8IL3,"[New Start, Don't Get Mad, Bussin, AM Track, I...","[{'danceability': 0.428, 'energy': 0.467, 'key...","{'danceability': 0.5502857142857143, 'energy':...",7,14,[]
4,stillness in wonderland,little simz,22704,7.1,4G50FUTTI4fCDyrxP1UEer,"[LMPD, Cheshire's Interlude: Welcome To Wonder...","[{'danceability': 0.614, 'energy': 0.507, 'key...","{'danceability': 0.5925333333333334, 'energy':...",15,46,[]


In [132]:
df2.to_pickle('df_pitchfork_spotify.pkl')